In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import glob
import os
import cv2
import time
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
names = pd.read_csv('../input/stanford-car-dataset-images-in-224x224/stanford-car-dataset-by-classes-folder-224/names.csv')
names = names.values
np.random.shuffle(names)

In [ ]:
nr_cars = 15
idx_to_name = {x : names[x][0] for x in np.arange(nr_cars)}
name_to_idx = {x:i for i,x in enumerate(idx_to_name.values())}
idx_to_name

In [ ]:
train_path = '../input/stanford-car-dataset-images-in-224x224/stanford-car-dataset-by-classes-folder-224/car_data/train/'
test_path = '../input/stanford-car-dataset-images-in-224x224/stanford-car-dataset-by-classes-folder-224/car_data/test/'

In [ ]:
def get_data(path):
    train = []
    for i, name in enumerate(name_to_idx.keys()):
        new_path = path + name + "/"
        [train.append([i, cv2.resize(cv2.imread(img), (244,244), interpolation = cv2.INTER_AREA)]) for img in glob.glob(new_path + "*.jpg")]
    return np.array(train)

In [ ]:
train = get_data(train_path)
test = get_data(test_path)

In [ ]:
X_tr = np.concatenate(train[:,1], axis=0).reshape(len(train), 244, 244, 3)
X_tr = X_tr / 255.0
X_tr = X_tr.astype('float32')
y_tr = train[:,0]
y_tr = np.eye(len(idx_to_name))[list(y_tr)]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size = 0.15, random_state = 42)
print("x_train shape = ",x_train.shape)
print("y_train shape = ",y_train.shape)
print("x_val shape = ",x_val.shape)
print("y_val shape = ",y_val.shape)

In [ ]:
model=Sequential(name="Alexnet")
#1 layer (conv + pool + batchnorm)
model.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="valid",activation="relu",input_shape=(227,227,3)))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(BatchNormalization())
#2 layer (conv + pool + batchnorm)
model.add(Conv2D(filters=256,kernel_size=(5,5),strides=(1,1),padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(BatchNormalization())
#3 layer (conv + batchnorm)
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))
model.add(BatchNormalization())
#4 layer (conv + batchnorm)
model.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))
model.add(BatchNormalization())
#5 layer (conv + pool + batchnorm)
model.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(BatchNormalization())
model.add(Flatten())
#1 dense layer
model.add(Dense(4096,input_shape=(227,227,3),activation="relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
#2 dense layer
model.add(Dense(4096,activation="relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
#3 dense layer
model.add(Dense(1000,activation="relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
#output layer
model.add(Dense(15,activation="softmax"))
model.summary()

In [ ]:
# reduce learning rate by 0.1 when the validation error plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1))
 
# set the SGD optimizer with lr of 0.01 and momentum of 0.9
optimizer = SGD(lr = 0.01, momentum = 0.9)
 
# compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# train the model
# call the reduce_lr value using callbacks in the training method
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_val, y_val))

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 100) # set the vertical range to [0-1]
plt.show()

In [ ]:
from functools import partial
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
 padding="SAME", use_bias=False)
class ResidualUnit(keras.layers.Layer):
 def __init__(self, filters, strides=1, activation="relu", **kwargs):
     super().__init__(**kwargs)
     self.activation = keras.activations.get(activation)
     self.main_layers = [
         DefaultConv2D(filters, strides=strides),
         keras.layers.BatchNormalization(),
         self.activation,
         DefaultConv2D(filters),
         keras.layers.BatchNormalization()]
     self.skip_layers = []
     if strides > 1:
         self.skip_layers = [
             DefaultConv2D(filters, kernel_size=1, strides=strides),
             keras.layers.BatchNormalization()]
def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
         Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
         skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)

In [ ]:
model2 = keras.models.Sequential(name="Resnet34")
model2.add(DefaultConv2D(64, kernel_size=7, strides=2,
 input_shape=[224, 224, 3]))
model2.add(keras.layers.BatchNormalization())
model2.add(keras.layers.Activation("relu"))
model2.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
 strides = 1 if filters == prev_filters else 2
 model2.add(ResidualUnit(filters, strides=strides))
 prev_filters = filters
model2.add(keras.layers.GlobalAvgPool2D())
model2.add(keras.layers.Flatten())
model2.add(keras.layers.Dense(15, activation="softmax"))
model2.summary()

In [ ]:
# reduce learning rate by 0.1 when the validation error plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1))
 
# set the SGD optimizer with lr of 0.01 and momentum of 0.9
optimizer = SGD(lr = 0.001, momentum = 0.9)
 # compile the model
model2.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model2.fit(x_train, y_train,
 validation_data=(x_val, y_val),
 epochs=20)

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 10) # set the vertical range to [0-1]
plt.show()